In [1]:
import os
import together
from typing import Any
from langchain.llms.base import LLM
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFDirectoryLoader, DirectoryLoader, PyPDFLoader, TextLoader
import os
import docx2pdf
import pptxtopdf
from fpdf import FPDF
import shutil

c:\Users\SZMik\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Para leer y guardar todos los archivos en PDF

### Txt > PDF 

In [2]:
def txt_to_pdf(nombre, ubicacion):
    
    # Crear objeto PDF
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    # Abrir el archivo de texto
    with open(nombre, "r", encoding='utf-8') as f:
        # Insertar el texto en el PDF usando multi_cell
        for x in f:
            pdf.multi_cell(0, 10, txt=x)

    # Guardar el PDF
    pdf.output(ubicacion + "/" + os.path.splitext(nombre)[0] + '.pdf')

### IMG > PDF

In [ ]:
!pip install pytesseract
import pytesseract
from PIL import Image

def extract_text_from_image(image_path):

    image = Image.open(image_path)
    
    text = pytesseract.image_to_string(image)
    
    return text
# Extraer texto de la imagen
image_path = "ley_lm.jpg"
output = extract_text_from_image(image_path)
print( output)

In [ ]:
!pip install reportlab

import pytesseract
from PIL import Image
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import os

def extract_text_from_image(image_path):
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image)
    return text

def save_text_to_pdf(text, pdf_path):
    c = canvas.Canvas(pdf_path, pagesize=letter)
    width, height = letter
    lines = text.split('\n')
    
    y = height - 40
    for line in lines:
        if y < 40:  # If we are near the bottom of the page
            c.showPage()
            y = height - 40
        c.drawString(40, y, line)
        y -= 14  # Move to the next line

    c.save()

# #Ejemplo
# image_path = 'agra.jpg'
# convert_image_to_pdf(image_path)


### Cualquier otro archivo (menos XLSX) > PDF

In [8]:
# PONER LA RUTA DE DONDE SE VAN A ***GUARDAR*** LOS ARCHIVOS EN PDF
output_folder = "docs"


# Para guardar el texto de la imagen en un archivo PDF 
def convert_image_to_pdf(image_path):
    text = extract_text_from_image(image_path)
    pdf_path = output_folder + '/' + os.path.splitext(image_path)[0] + '.pdf'
    save_text_to_pdf(text, pdf_path)
    print(f"Texto almacenadoe en  {pdf_path}")


# PONER LA RUTA DE DONDE SE VAN A ***LEER*** LOS ARCHIVOS ORIGINALES
ejemplo_dir = '.'

with os.scandir(ejemplo_dir) as ficheros:
    for fichero in ficheros:
        input_file = fichero.name

        if ((fichero.name[-3] == 'o') & (fichero.name[-2] == 'c') & (fichero.name[-1] == 'x')): #docx
            docx2pdf.convert(input_file, output_folder)

        if ((fichero.name[-3] == 'p') & (fichero.name[-2] == 't') & (fichero.name[-1] == 'x')): #pptx
            pptxtopdf.convert(input_file, output_folder)

        if ((fichero.name[-3] == 't') & (fichero.name[-2] == 'x') & (fichero.name[-1] == 't')): #txt
            txt_to_pdf(input_file, output_folder)

        if ((fichero.name[-3] == 'p') & (fichero.name[-2] == 'd') & (fichero.name[-1] == 'f')): #pdf
            shutil.copy2(input_file, output_folder)

        if ((fichero.name[-3] == 'j') & (fichero.name[-2] == 'p') & (fichero.name[-1] == 'g')): #jpg???
            convert_image_to_pdf(input_file)

100%|██████████| 1/1 [00:03<00:00,  3.61s/it]


Conversion completed: 1 files converted successfully, 0 files failed.
Conversion completed: 1 files converted successfully, 0 files failed.


**NO TOCAR!**

In [ ]:
os.environ["TOGETHER_API_KEY"] = "f8935229473a0d8a3f4709a9ef32533fe365c0cb215ba8c41413b5ca53a5c767"

class TogetherLLM(LLM):
    model: str = "togethercomputer/llama-2-7b-chat"
    temperature: float = 0.1
    max_tokens: int = 1024
    together_api_key: str = os.environ.get("TOGETHER_API_KEY")
    
    class Config:
        extra = 'forbid'
    @property
    def _llm_type(self) -> str:
        return "together"
    def _call(self, prompt: str, **kwargs: Any) -> str:
        if not self.together_api_key:
            raise ValueError("API key is not set.")
        together.api_key = self.together_api_key
        output = together.Complete.create(prompt,
                                        model=self.model,
                                        max_tokens=self.max_tokens,
                                        temperature=self.temperature)
        print(f"modelo: {self.model}")
        if 'choices' in output:
            return output['choices'][0]['text']
        else:
            raise KeyError("The key 'choices' is not in the response.")

# # Funcion para traduccion a ingles
# from googletrans import Translator
# import googletrans
# translator = Translator()

# def traducir(texto_original):
#     try :
#         origen = translator.detect(texto_original).lang
#         # Si el texto esta en otro idioma que no sea ingles lo traduce
#         if origen != "en":
#             #print(f'Traduccion de {origen} a en')
#             traduccion = translator.translate(texto_original, dest="en", src=origen).text
#             return traduccion
#         # En caso contrario devuelve el texto original ya que esta en ingles
#         else:
#             return texto_original
#     except:
#         return texto_original
    
# # Funcion para traduccion a otro idioma
# def traducir_ingles(texto_ingles, idioma_destino):
#     for abreviatura, nombre_idioma in googletrans.LANGUAGES.items():
#         if nombre_idioma == idioma_destino:
#             destino = abreviatura
#     traduccion = translator.translate(texto_ingles, dest=destino, src="en").text
#     return traduccion

# def dividir_texto(texto, max_caracteres):
#     textos_divididos = []
#     texto_actual = ''
#     caracteres_actuales = 0

#     oraciones = texto.split('.')

#     for oracion in oraciones:
#         caracteres_actuales += len(oracion) + 1

#         if caracteres_actuales <= max_caracteres:
#             texto_actual += oracion + '.'
#         else:
#             textos_divididos.append(texto_actual.strip())
#             texto_actual = oracion + '.'
#             caracteres_actuales = len(oracion) + 1

#     textos_divididos.append(texto_actual.strip())

#     return textos_divididos

def get_prompt(instruction, new_system_prompt ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template



B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

sys_prompt = """You are a helpful, respectful, and honest assistant. Always provide the most helpful and accurate answer using only the contextual text provided. Do not add any information that is not in the context.

Guidelines:

1. **Relevance**: Only answer questions based on the provided context. Do not use outside knowledge.
2. **Honesty**: If a question does not make sense or is factually incoherent, explain why instead of providing incorrect information.
3. **Integrity**: If you don't know the answer based on the context, do not share false information.
4. **Scope**: If the question is outside the context, inform the user politely that it cannot be answered accurately.
5. **Clarity**: Ensure your answers are clear and concise, avoiding ambiguity or vagueness.
6. **Finality**: Answer the question directly and do not include any additional text after the answer.

Example of handling an irrelevant question:
- "I'm sorry, but this question is outside the provided context and I cannot answer it accurately."

Example of handling a nonsensical question:
- "The question seems factually incoherent, so I cannot provide an accurate answer."

Ensure that your answers are clear and concise, avoiding ambiguity or vague responses."""

instruction = """CONTEXT:/n/n {context}/n

Question: {question}"""

prompt_template = get_prompt(instruction, sys_prompt)
llama_prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": llama_prompt}

def modelo_llm(modelo):
    return TogetherLLM(
        model= modelo,
    )


In [10]:
def load_documents(directory):
    # documentPDF = PyPDFDirectoryLoader(directory)
    documentPDF = DirectoryLoader(directory, glob='./*.pdf', loader_cls=PyPDFLoader)
    # documentPDF = DirectoryLoader('./documents', glob='./*.txt', loader_cls=TextLoader)
    documents = documentPDF.load()
    return documents

def chunk_data(docs, chunk_size=800, overlap=100):
    text_spliter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=overlap
        )
    return text_spliter.split_documents(docs)


documents = load_documents('docs')
print(f"tamano de documentos: {len(documents)}")
chunks = chunk_data(documents)
print(f"tamano de chunks: {len(chunks)}")

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

Ignoring wrong pointing object 26 0 (offset 0)
Ignoring wrong pointing object 29 0 (offset 0)
Ignoring wrong pointing object 34 0 (offset 0)
Ignoring wrong pointing object 36 0 (offset 0)
Ignoring wrong pointing object 38 0 (offset 0)


tamano de documentos: 65
tamano de chunks: 128


c:\Users\SZMik\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Si se desea usar Pinecone:

In [ ]:
# from pinecone import Pinecone, ServerlessSpec
# from langchain_pinecone import PineconeVectorStore

# os.environ['PINECONE_API_KEY'] = '57210699-f119-4a04-9224-30521bbd7dc3'

# index_name = "chatify-384"

# pinecone = PineconeVectorStore.from_documents(
#     chunks, embeddings, index_name=index_name
# )

Choma DB

In [ ]:
%pip install chromadb

# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
from langchain.vectorstores import Chroma
persist_directory = 'db'

vectordb = Chroma.from_documents(documents=chunks, 
                                 embedding=embeddings,
                                 persist_directory=persist_directory)

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [ ]:
retriever.search_kwargs

In [ ]:
modelo = "togethercomputer/llama-2-7b-chat"
llm = modelo_llm(modelo)
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                    chain_type="stuff",
                                    retriever=retriever,
                                    return_source_documents=True,
                                    chain_type_kwargs=chain_type_kwargs)

In [ ]:
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nFuentes:')
    for source in llm_response["source_documents"]:
        # Acceder a la metadata para obtener la página
        page_number = source.metadata['page']
        print(f"Fuente: {source.metadata['source']}, Pagina: {page_number}")


In [ ]:
query = "Que es la SS"
llm_response = qa_chain(query)
process_llm_response(llm_response)

In [ ]:
# def conversation_chat(query,chain):
#     #result =chain({"question":query, "chat_history": history})
#     user_question_eng = traducir(query)
#     llm_response = chain.invoke(user_question_eng)
#     llm_response = traducir_ingles(llm_response['result'], "spanish")
#     return llm_response

In [ ]:
# pdf_obj = "./historia.pdf"
# knowledge_base = create_embeddings(pdf_obj)

# retriever = knowledge_base.as_retriever(search_kwargs={"k": 3})
# modelo = "togethercomputer/llama-2-7b-chat"
# llm = modelo_llm(modelo)
# qa_chain = RetrievalQA.from_chain_type(llm=llm,
#                                     chain_type="stuff",
#                                     retriever=retriever,
#                                     chain_type_kwargs=chain_type_kwargs)

# user_question = "De que trata el documento?"
# conversation_chat(user_question, qa_chain )